<a href="https://colab.research.google.com/github/senakrgli/MEPs/blob/main/MEPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import openpyxl

# Web sayfasını talep et

In [ ]:
url = "https://www.europarl.europa.eu/meps/en/full-list/all"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# CSV dosyası için başlık tanımları

In [ ]:
header = ['URL', 'Name', 'Birthday', 'Birthplace', 'Political Group', 'Political Group Role', 'Country',
          'National Party', 'Email', 'Website', 'Twitter', 'Instagram', 'Facebook', 'Bruxelles Contact',
          'Strasbourg Contact', 'Assistants', 'Chair (Committees and delegations)',
          'Member (Committees and delegations)', 'Substitute (Committees and delegations)',
          'Vice-Chair (Committees and delegations)']

# Verileri saklamak için boş bir liste oluştur

In [ ]:
meps_data = []

#Her bir üye için HTML parçalarını bul, 705 üye için belirlediğin değişkenlere gerekenleri al

In [ ]:
mep_members = soup.find_all(name="a", class_="erpl_member-list-item-content")
for member in mep_members:
    first_member_href = member.get('href')
    href_url = requests.get(first_member_href)
    href_soup = BeautifulSoup(href_url.text, "html.parser")

    URL = member.get('href')

    Name = member.find(name="div", class_="erpl_title-h4").text.strip()

    birth_date = href_soup.find(name="time", class_="sln-birth-date", )

    birth_date_text = ""
    if birth_date:
        birth_date_text = birth_date.get("datetime")

    Birthplace = href_soup.find(name="span", class_="sln-birth-place")
    birth_place_text = ""
    if Birthplace:
        birth_place_text = Birthplace.get_text()

    Political_Group = href_soup.find(name="h3", class_="erpl_title-h3 mt-1 sln-political-group-name")
    Group_text = ""
    if Political_Group:
        Group_text = Political_Group.get_text()

    Political_Group_Role = href_soup.find(name="p", class_="sln-political-group-role")
    Role_text = ""
    if Political_Group_Role:
        Role_text = Political_Group_Role.get_text()

    Country = href_soup.find(name="div", class_="erpl_title-h3 mt-1 mb-1")
    Country_text = ""
    if len(Country.get_text().split("-")) > 0:
        Country_text = Country.get_text().split("-")[0].strip()

    National_Party = href_soup.find(name="div", class_="erpl_title-h3 mt-1 mb-1")
    National_text = ""
    if len(National_Party.get_text().split("-")) > 1:
        National_text = National_Party.get_text().split("-")[1].strip()

    Email = href_soup.find(name="a", class_="link_email mr-2")
    email_href_text = ""
    if Email:
        email_href = Email.get("href")
        if email_href:
            email_parts = email_href.split(":")
            if len(email_parts) > 1:
                email_href_text = email_parts[1].strip()

    Website = href_soup.find(name="a", class_="link_website mr-2")
    website_href = ""
    if Website:
        website_href = Website.get("href")

    Twitter = href_soup.find(name="a", class_="link_twitt mr-2")
    Twitter_href = ""
    if Twitter:
        Twitter_href = Twitter.get("href")

    Instagram = href_soup.find(name="a", class_="link_instagram mr-2")
    Instagram_href = ""
    if Instagram:
        Instagram_href = Instagram.get("href")

    Facebook =href_soup.find(name="a", class_="link_fb mr-2")
    Facebook_href = ""
    if Facebook:
        Facebook_href = Facebook.get("href")

    Bruxelles_Contact = href_soup.find(name="div", class_="erpl_contact-card-list")
    b_contact = ""
    if Bruxelles_Contact:
        b_contact = Bruxelles_Contact.get_text().replace('\n', '')

    Strasbourg_Contact_list = href_soup.find_all(name="div", class_="erpl_contact-card-list")
    s_contact = ""
    if len(Strasbourg_Contact_list) > 1:
        Strasbourg_Contact = Strasbourg_Contact_list[1]
        s_contact = Strasbourg_Contact.get_text().replace('\n','')

    Chair = ""
    Member = ""
    Substitute = ""
    ViceChair = ""
    tip = ""
    erpl_titles = href_soup.find_all(name="div", class_="erpl_meps-status")
    for erpl_title in erpl_titles:
        tip = erpl_title.find(name ="h4", class_="erpl_title-h4").get_text()
        commiteler = erpl_title.find_all(name = "div", class_= "erpl_committee")
        commiteString = ""
        for commite in commiteler:
            commiteString += commite.get_text() + ', '
        if(tip == 'Member'):
             Member = commiteString
        elif(tip == 'Substitute'):
            Substitute = commiteString
        elif(tip ==  'Chair'):
            Chair = commiteString
        elif('Vice-Chair'):
            ViceChair = commiteString

    assistant_href_tag = href_soup.find(string='Assistants', class_= "t-x")
    assistant_href = assistant_href_tag.find_parent(name = "a", class_ = 'erpl_accordion-item-title').get('href')
    href_as_url = requests.get('https://www.europarl.europa.eu' + assistant_href)
    href_as_soup = BeautifulSoup(href_as_url.text, "html.parser")

    Assistant_list = href_as_soup.find_all(name='span', class_= 'erpl_assistant')
    Assistants = ""
    for assistant in Assistant_list:
        Assistants += assistant.get_text().strip() + ", "

    meps_data.append([URL, Name, birth_date_text, birth_place_text, Group_text, Role_text, Country_text,
              National_text, email_href_text, website_href, Twitter_href, Instagram_href, Facebook_href,
              b_contact, s_contact, Assistants, Chair, Member, Substitute, ViceChair])

#Bir .xlsx dosyası oluştur

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.active


header = ['URL', 'Name', 'Birthday', 'Birthplace', 'Political Group', 'Political Group Role', 'Country',
          'National Party', 'Email', 'Website', 'Twitter', 'Instagram', 'Facebook', 'Bruxelles Contact',
          'Strasbourg Contact', 'Assistants', 'Chair (Committees and delegations)',
          'Member (Committees and delegations)', 'Substitute (Committees and delegations)',
          'Vice-Chair (Committees and delegations)']


sheet.append(header)

for member_data in meps_data:
    sheet.append(member_data)

workbook.save("meps_data.xlsx")